In [1]:
import random
import os
os.environ["OMP_NUM_THREADS"] = "1"
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
import cv2
import warnings
import timeit
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from torch import optim
from scipy.special import expit
from skimage import io, transform, measure
from sklearn import metrics
import optuna
from config import *
from util import *
from models import *
sns.set()
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
rc = {"figure.figsize" : (9, 7),
      "axes.spines.left" : False,
      "axes.spines.right" : False,
      "axes.spines.bottom" : False,
      "axes.spines.top" : False,
      "xtick.bottom" : True,
      "xtick.labelbottom" : False,
      "ytick.labelleft" : False,
      "ytick.left" : True,
      "axes.grid" : False}
plt.rcParams.update(rc)

In [2]:

torch.cuda.empty_cache()
random.seed(seed)

params = {'batch_size': 16, 'dropout': 0.509903, 'learning_rate': 0.000033}

checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor="val/loss")
model = BaselineClassifier(img_dir=img_dir, metadata_file=metadata_file, weighted_sampling=False,
                           batch_size=params['batch_size'], dropout=params['dropout'], learning_rate=params['learning_rate'])

trainer = pl.Trainer(max_epochs=25, devices=[1], accelerator="gpu", deterministic=True, callbacks=[checkpoint_callback], weights_summary=None)
trainer.fit(model)

# Load the best checkpoint after training
best_checkpoint_path = checkpoint_callback.best_model_path
checkpoint = torch.load(best_checkpoint_path)
model.load_state_dict(checkpoint['state_dict'])


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

<All keys matched successfully>

In [3]:
result_val = get_predictions(trainer, model, split='val', dx_threshold=0)
print('Validation:')
display_metrics(result_val)
dx_threshold = find_optimal_cutoff(result_val['benign_malignant'], result_val['dx_score'])
result_test = get_predictions(trainer, model, split='test', dx_threshold=dx_threshold)
print('Test:')
display_metrics(result_test)
result_external = get_predictions(trainer, model, split='external', dx_threshold=dx_threshold) 
print('External:')
display_metrics(result_external)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting: 169it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Validation:
balanced acc:  0.8246
sensitivity:  0.82759
specificity:  0.82162


Predicting: 169it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Test:
balanced acc:  0.83619
sensitivity:  0.792
specificity:  0.88038


Predicting: 169it [00:00, ?it/s]

External:
balanced acc:  0.77134
sensitivity:  0.72449
specificity:  0.81818


In [4]:
dx_threshold

0.018226001411676407

In [5]:
#torch.save(model.state_dict(), os.path.join("models", "baseline_model.pth"))

In [6]:
display_metrics(result_test[result_test.lesion_id.str.contains('HAM')])

balanced acc:  0.79564
sensitivity:  0.70312
specificity:  0.88816


In [7]:
result_val = get_predictions(trainer, model, split='val', dx_threshold=0)
print('Validation:')
display_metrics(result_val)
dx_threshold = find_optimal_cutoff(result_val['benign_malignant'], result_val['dx_score'])
result_test = get_predictions(trainer, model, split='test', dx_threshold=dx_threshold)
print('Test:')
display_metrics(result_test)
result_external = get_predictions(trainer, model, split='external', dx_threshold=dx_threshold) 
print('External:')
display_metrics(result_external)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Predicting: 169it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Validation:
balanced acc:  0.8246
sensitivity:  0.82759
specificity:  0.82162


Predicting: 169it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Test:
balanced acc:  0.83619
sensitivity:  0.792
specificity:  0.88038


Predicting: 169it [00:00, ?it/s]

External:
balanced acc:  0.77134
sensitivity:  0.72449
specificity:  0.81818


In [9]:
result_val.to_csv(os.path.join("results", "result_val.csv"), index=False)
result_test.to_csv(os.path.join("results", "result_test.csv"), index=False)
result_external.to_csv(os.path.join("results", "result_external.csv"), index=False)